In [ ]:
def read_uint64_pointer(val, count):
    # val: gdb.Value pointing to uint64_t *
    # count: number of elements to read
    inf = gdb.selected_inferior()
    address = int(val)
    length = count * 8  # 8 bytes per uint64_t
    mem = inf.read_memory(address, length)
    # Convert bytes to list of uint64_t (big-endian)
    result = [int.from_bytes(mem[i*8:(i+1)*8], 'big') for i in range(count)]
    return result

In [2]:
def read_bits(words, bit_offset, num_bits):
    # words: list of 64-bit integers
    # bit_offset: starting bit position (0 = MSB of words[0])
    # num_bits: number of bits to read
    result = 0
    for i in range(num_bits):
        pos = bit_offset + i
        word_idx = pos // 64
        bit_idx = pos % 64
        # MSB is bit 0, so shift by (63 - bit_idx)
        bit = (words[word_idx] >> (63 - bit_idx)) & 1
        result = (result << 1) | bit
    return result

In [3]:
# Test read_bits function
words = [0xF0F0F0F0F0F0F0F0, 0x123456789ABCDEF0]
bit_offset = 60
num_bits = 8
result = read_bits(words, bit_offset, num_bits)
print(f"Bits from offset {bit_offset} (next {num_bits} bits): {bin(result)} ({result})")

# Test reading across word boundary
bit_offset2 = 62
num_bits2 = 6
result2 = read_bits(words, bit_offset2, num_bits2)
print(f"Bits from offset {bit_offset2} (next {num_bits2} bits): {bin(result2)} ({result2})")

Bits from offset 60 (next 8 bits): 0b1 (1)
Bits from offset 62 (next 6 bits): 0b1 (1)


In [5]:
# Read 12 bits from offset 56
bit_offset3 = 56
num_bits3 = 12
result3 = read_bits(words, bit_offset3, num_bits3)
print(f"Bits from offset {bit_offset3} (next {num_bits3} bits): {bin(result3)} ({result3}), hex: {hex(result3)}")

Bits from offset 56 (next 12 bits): 0b111100000001 (3841), hex: 0xf01


## How to call a member function of a C++ object with gdb.Value in GDB Python
To call a member function of an object using a `gdb.Value`, use the `gdb.parse_and_eval` function with the appropriate C++ expression as a string.

**Example:**
```python
# If obj is a gdb.Value representing an object (not a pointer):
result = gdb.parse_and_eval(f'({obj.type.pointer()}){int(obj.address)}->foo()')

# If obj is a pointer already:
result = gdb.parse_and_eval(f'{obj}.foo()')

# Or using the object's name if available:
result = gdb.parse_and_eval('obj.foo()')
```
This will evaluate the member function and return a new `gdb.Value` with the result.

## Accessing static constexpr members of a class in GDB Python
To access a `static constexpr` member of a class, use `gdb.parse_and_eval` with the fully qualified name:
```python
value = gdb.parse_and_eval('ClassName::constexpr_member')
```
Replace `ClassName` and `constexpr_member` with your actual class and member names.
This works for both `static constexpr` and `static const` members, as long as they are accessible in the debug info. No object instance is needed.

In [ ]:
import traceback

try:
    # ... your code ...
except Exception as e:
    tb = traceback.extract_tb(e.__traceback__)
    last_entry = tb[-1]
    print(f"Exception at {last_entry.filename}, line {last_entry.lineno}: {last_entry.line}")

## Reading a uint64_t[] array with gdb.Value in GDB Python
To read a `uint64_t[]` array using a `gdb.Value`, index into the array and convert each element to a Python integer:
```python
def read_uint64_array(val, count):
    # val: gdb.Value pointing to uint64_t[]
    # count: number of elements to read
    result = [int(val[i]) for i in range(count)]
    return result
```
This returns a Python list of integers from the `uint64_t` array.

## Getting the real type of gdb.Value (removing references and const modifiers)

The best way to get the underlying type of a `gdb.Value` by removing reference and const modifiers is to use a combination of type operations:

```python
def real_type(val):
    """
    Get the real underlying type of a GDB value.
    
    This function processes a GDB value to extract its actual type by:
    1. Getting the initial type from the value
    2. Dereferencing if it's a reference type  
    3. Removing qualifiers (const, volatile) and typedef aliases
    
    Args:
        val: A GDB value object whose type needs to be determined
        
    Returns:
        gdb.Type: The unqualified, stripped type without references or typedefs
    """
    # Get the type.
    type = val.type

    # If it points to a reference, get the reference.
    if type.code == gdb.TYPE_CODE_REF:
        type = type.target()

    # Get the unqualified type, stripped of typedefs.
    type = type.unqualified().strip_typedefs()

    return type
```

**How it works:**
1. `val.type` - Gets the initial type from the GDB value
2. **Reference handling** - If the type is a reference (`gdb.TYPE_CODE_REF`), it calls `type.target()` to get the target type
3. **Qualifier removal** - `type.unqualified()` removes const, volatile, and other qualifiers
4. **Typedef stripping** - `strip_typedefs()` resolves any typedef aliases to get the actual underlying type

**Examples:**
- For `const std::vector<int>&` → returns `std::vector<int>`
- For `const MyTypedef&` where MyTypedef is `int` → returns `int`

## GitHub Examples of GDB Pretty Printers

Here are some excellent real-world examples of GDB pretty printers from major projects:

### Repository Links

- **LLVM Project**: https://github.com/llvm/llvm-project/tree/main/llvm/utils/gdb-scripts/prettyprinters.py
- **libc++**: https://github.com/llvm/llvm-project/tree/main/libcxx/utils/gdb/libcxx/printers.py  
- **GCC**: https://github.com/gcc-mirror/gcc/tree/main/gcc/gdbhooks.py
- **Boost (Rüdiger)**: https://github.com/ruediger/Boost-Pretty-Printer
- **Boost (Alternative)**: https://github.com/mateidavid/Boost-Pretty-Printer

### LLVM Project Examples

**1. SmallVector Printer (llvm/utils/gdb-scripts/prettyprinters.py)**
```python
class SmallVectorPrinter(Iterator):
    """Print an llvm::SmallVector object."""

    def __init__(self, val):
        self.val = val
        t = val.type.template_argument(0).pointer()
        self.begin = val["BeginX"].cast(t)
        self.size = val["Size"]
        self.i = 0

    def __next__(self):
        if self.i == self.size:
            raise StopIteration
        ret = "[{}]".format(self.i), (self.begin + self.i).dereference()
        self.i += 1
        return ret

    def to_string(self):
        return "llvm::SmallVector of Size {}, Capacity {}".format(
            self.size, self.val["Capacity"]
        )

    def display_hint(self):
        return "array"
```

**2. StringRef Printer**
```python
class StringRefPrinter:
    """Print an llvm::StringRef object."""

    def __init__(self, val):
        self.val = val

    def to_string(self):
        data = self.val["Data"]
        length = self.val["Length"]
        return data.lazy_string(length=length)

    def display_hint(self):
        return "string"
```

**3. DenseMap Printer (with complex iterator)**
```python
class DenseMapPrinter:
    "Print a DenseMap"

    def __init__(self, val):
        self.val = val

    def children(self):
        t = self.val.type.template_argument(3).pointer()
        begin = self.val["Buckets"].cast(t)
        end = (begin + self.val["NumBuckets"]).cast(t)
        return self._iterator(self.val.type.template_argument(2), begin, end)

    def to_string(self):
        return "llvm::DenseMap with %d elements" % (self.val["NumEntries"])

    def display_hint(self):
        return "map"
```

### libc++ Examples

**1. std::string Printer**
```python
class StdStringPrinter(object):
    """Print a std::string."""

    def __init__(self, val):
        self.val = val

    def to_string(self):
        value_field = self.val["__rep_"]
        short_field = value_field["__s"]
        if short_field["__is_long_"]:
            long_field = value_field["__l"]
            data = long_field["__data_"]
            size = long_field["__size_"]
        else:
            data = short_field["__data_"]
            size = short_field["__size_"]
        return data.lazy_string(length=size)

    def display_hint(self):
        return "string"
```

**2. std::shared_ptr Printer**
```python
class StdSharedPointerPrinter(object):
    """Print a std::shared_ptr."""

    def __init__(self, val):
        # Implementation details...

    def to_string(self):
        """Returns self as a string."""
        # Return formatted string representation

    def children(self):
        # Yield pointer and reference count info
        yield ("ptr", self.pointer)
        yield ("use_count", self.use_count)
```

### GCC Internal Examples

**1. VecPrinter (gcc/gdbhooks.py)**
```python
class VecPrinter:
    def __init__(self, gdbval):
        self.gdbval = gdbval

    def display_hint(self):
        return 'array'

    def to_string(self):
        return '0x%x' % intptr(strip_ref(self.gdbval))

    def children(self):
        # Implementation for iterating over vector elements
```

### Key Patterns from GitHub Examples

**Essential Methods:**
1. `__init__(self, val)` - Initialize with gdb.Value
2. `to_string(self)` - Return string representation  
3. `children(self)` - Return iterator for sub-elements
4. `display_hint(self)` - Return display type ("array", "map", "string")

**Common Display Hints:**
- `"array"` - For containers like vector, list, SmallVector
- `"map"` - For associative containers like map, DenseMap, StringMap  
- `"string"` - For string-like objects like StringRef, string

**Registration Pattern:**
```python
pp = gdb.printing.RegexpCollectionPrettyPrinter("project_name")
pp.add_printer("TypeName", "^namespace::TypeName<.*>$", PrinterClass)
gdb.printing.register_pretty_printer(gdb.current_objfile(), pp)
```

### Complete Example Template

```python
class MyTypePrinter:
    """Print a MyType object."""
    
    def __init__(self, val):
        self.val = val
        # Extract and cache commonly used fields
        self.data = val["data"]
        self.size = int(val["size"])
    
    def to_string(self):
        # Return brief description
        return f"MyType with {self.size} elements"
    
    def children(self):
        # Iterate over sub-elements
        for i in range(self.size):
            yield f"[{i}]", self.data[i]
    
    def display_hint(self):
        # Help GDB format the output appropriately
        return "array"  # or "map" or "string"

# Registration
pp = gdb.printing.RegexpCollectionPrettyPrinter("my_project")
pp.add_printer("MyType", "^MyNamespace::MyType<.*>$", MyTypePrinter)
gdb.printing.register_pretty_printer(gdb.current_objfile(), pp)
```

## Boost Pretty Printer Examples

Boost library also has excellent pretty printer implementations that demonstrate advanced techniques.

### Repository Links

- **Rüdiger's Boost Pretty Printer**: https://github.com/ruediger/Boost-Pretty-Printer
- **Alternative Boost Pretty Printer**: https://github.com/mateidavid/Boost-Pretty-Printer

### Key Examples from Boost Pretty Printers

**1. boost::optional Printer**
```python
class BoostOptionalPrinter:
    """Pretty printer for boost::optional"""
    
    def __init__(self, val):
        self.val = val
        self.contained_type = val.type.template_argument(0)
    
    def to_string(self):
        if self.val['m_initialized']:
            return f"boost::optional<{self.contained_type}> [initialized]"
        else:
            return f"boost::optional<{self.contained_type}> [uninitialized]"
    
    def children(self):
        if self.val['m_initialized']:
            # Access the stored value  
            storage = self.val['m_storage']
            value = storage['dummy_']['data'].cast(self.contained_type)
            yield ('value', value)
```

**2. boost::shared_ptr Printer**
```python
class BoostSharedPtrPrinter:
    """Pretty printer for boost::shared_ptr"""
    
    def __init__(self, val):
        self.val = val
        self.pointee_type = val.type.template_argument(0)
    
    def to_string(self):
        px = self.val['px']
        if px == 0:
            return f"boost::shared_ptr<{self.pointee_type}> [empty]"
        else:
            use_count = self.val['pn']['pi_']['use_count_']
            return f"boost::shared_ptr<{self.pointee_type}> [use_count={use_count}]"
    
    def children(self):
        px = self.val['px']
        if px != 0:
            yield ('px', px.dereference())
            yield ('use_count', self.val['pn']['pi_']['use_count_'])
            yield ('weak_count', self.val['pn']['pi_']['weak_count_'])
```

**3. boost::circular_buffer Printer**
```python
class BoostCircularBufferPrinter:
    """Pretty printer for boost::circular_buffer"""
    
    def __init__(self, val):
        self.val = val
        self.element_type = val.type.template_argument(0)
    
    def to_string(self):
        size = self.val['m_size']
        capacity = self.val['m_end'] - self.val['m_buff']
        return f"boost::circular_buffer<{self.element_type}> [size={size}, capacity={capacity}]"
    
    def children(self):
        begin = self.val['m_first']
        end = self.val['m_end']
        last = self.val['m_last']
        size = int(self.val['m_size'])
        
        # Handle circular nature of the buffer
        for i in range(size):
            element_ptr = begin + i
            if element_ptr >= end:
                element_ptr = self.val['m_buff'] + (element_ptr - end)
            yield (f'[{i}]', element_ptr.dereference())
    
    def display_hint(self):
        return 'array'
```

**4. boost::array Printer**
```python
class BoostArrayPrinter:
    """Pretty printer for boost::array"""
    
    def __init__(self, val):
        self.val = val
        self.element_type = val.type.template_argument(0)
        self.size = val.type.template_argument(1)
    
    def to_string(self):
        return f"boost::array<{self.element_type}, {self.size}>"
    
    def children(self):
        elems = self.val['elems']
        for i in range(int(self.size)):
            yield (f'[{i}]', elems[i])
    
    def display_hint(self):
        return 'array'
```

**5. boost::unordered_map Printer**
```python
class BoostUnorderedMapPrinter:
    """Pretty printer for boost::unordered_map"""
    
    def __init__(self, val):
        self.val = val
        self.key_type = val.type.template_argument(0)
        self.value_type = val.type.template_argument(1)
    
    def to_string(self):
        size = self.val['table_']['size_']
        return f"boost::unordered_map<{self.key_type}, {self.value_type}> [size={size}]"
    
    def children(self):
        table = self.val['table_']
        buckets = table['buckets_']
        bucket_count = table['bucket_count_']
        
        for i in range(int(bucket_count)):
            bucket = buckets[i]
            node = bucket['next_']
            while node != 0:
                # Extract key-value pair from node
                value_ptr = node + 1  # Assuming node layout
                key = value_ptr['first']
                value = value_ptr['second']
                yield (str(key), value)
                node = node['next_']
    
    def display_hint(self):
        return 'map'
```

**6. boost::intrusive::list Printer**
```python
class BoostIntrusiveListPrinter:
    """Pretty printer for boost::intrusive::list"""
    
    def __init__(self, val):
        self.val = val
        self.element_type = val.type.template_argument(0)
    
    def to_string(self):
        # Count elements by traversing the list
        count = 0
        node = self.val['data_']['node_plus_']['node_']
        first_node = node
        node = node['next_']
        
        while node != first_node:
            count += 1
            node = node['next_']
        
        return f"boost::intrusive::list<{self.element_type}> [size={count}]"
    
    def children(self):
        node = self.val['data_']['node_plus_']['node_']
        first_node = node
        node = node['next_']
        index = 0
        
        while node != first_node:
            # Cast node to actual element type
            element = node.cast(self.element_type.pointer()).dereference()
            yield (f'[{index}]', element)
            node = node['next_']
            index += 1
    
    def display_hint(self):
        return 'array'
```

### Registration Pattern for Boost Types

```python
def register_boost_printers():
    """Register all boost pretty printers"""
    pp = gdb.printing.RegexpCollectionPrettyPrinter("boost")
    
    # Optional
    pp.add_printer('boost::optional', 
                   '^boost::optional<.*>$', 
                   BoostOptionalPrinter)
    
    # Smart pointers
    pp.add_printer('boost::shared_ptr', 
                   '^boost::shared_ptr<.*>$', 
                   BoostSharedPtrPrinter)
    
    # Containers
    pp.add_printer('boost::circular_buffer', 
                   '^boost::circular_buffer<.*>$', 
                   BoostCircularBufferPrinter)
    
    pp.add_printer('boost::array', 
                   '^boost::array<.*>$', 
                   BoostArrayPrinter)
    
    pp.add_printer('boost::unordered_map', 
                   '^boost::unordered_map<.*>$', 
                   BoostUnorderedMapPrinter)
    
    pp.add_printer('boost::intrusive::list', 
                   '^boost::intrusive::list<.*>$', 
                   BoostIntrusiveListPrinter)
    
    gdb.printing.register_pretty_printer(gdb.current_objfile(), pp)

# Auto-register when module is loaded
register_boost_printers()
```

### Key Techniques from Boost Examples

**1. Template Argument Access:**
```python
# Get template arguments
element_type = val.type.template_argument(0)
size_value = val.type.template_argument(1)
```

**2. Pointer Arithmetic:**
```python
# Navigate pointer structures
element_ptr = begin + i
if element_ptr >= end:
    element_ptr = buffer_start + (element_ptr - end)
```

**3. Null Pointer Checking:**
```python
px = self.val['px']
if px == 0:
    return "empty"
else:
    return px.dereference()
```

**4. Circular Buffer Handling:**
```python
# Handle wraparound in circular buffers
for i in range(size):
    element_ptr = begin + i
    if element_ptr >= end:
        element_ptr = buffer_start + (element_ptr - end)
```

**5. Intrusive Container Navigation:**
```python
# Walk linked list structures
node = first_node['next_']
while node != first_node:
    element = node.cast(element_type.pointer()).dereference()
    yield element
    node = node['next_']
```

These Boost examples demonstrate sophisticated pretty printing techniques for complex C++ data structures involving template metaprogramming, intrusive containers, and smart pointer implementations.

For the main examples section:

LLVM Project: https://github.com/llvm/llvm-project/tree/main/llvm/utils/gdb-scripts/prettyprinters.py
libc++: https://github.com/llvm/llvm-project/tree/main/libcxx/utils/gdb/libcxx/printers.py
GCC: https://github.com/gcc-mirror/gcc/tree/main/gcc/gdbhooks.py
For the Boost examples section:

Rüdiger's Boost Pretty Printer: https://github.com/ruediger/Boost-Pretty-Printer
Alternative Boost Pretty Printer: https://github.com/mateidavid/Boost-Pretty-Printer
Now you have direct links to all the repositories where these examples come from, making it easy to explore the full source code and see how these pretty printers are implemented in their actual context. These are all production-quality implementations that you can use as references for your own GDB pretty printer development.

## Debugging Tips for Pretty Printer Development

### Common Issues and Solutions

**1. Pretty Printer Not Loading**
```python
# Check if your printer is registered
(gdb) info pretty-printer
# Should show your printer in the list

# Manual registration for testing
(gdb) python exec(open('/path/to/your_printer.py').read())

# Check for Python errors
(gdb) set python print-stack full
```

**2. AttributeError: No such field**
```python
# Debug field access issues
def safe_field_access(val, field_name):
    try:
        return val[field_name]
    except gdb.error as e:
        print(f"Field '{field_name}' not found in {val.type}")
        print(f"Available fields: {[f.name for f in val.type.fields()]}")
        raise e
```

**3. Type Mismatch Issues**
```python
# Robust type checking
def check_type_compatibility(val, expected_typename):
    actual_type = real_type(val)
    if expected_typename not in str(actual_type):
        print(f"Type mismatch: expected {expected_typename}, got {actual_type}")
        return False
    return True
```

**4. Template Argument Access Failures**
```python
# Safe template argument access
def safe_template_arg(val, index):
    try:
        return val.type.template_argument(index)
    except RuntimeError:
        print(f"Template argument {index} not available for {val.type}")
        return None
```

### Debug Output Techniques

**1. Conditional Debug Printing**
```python
DEBUG_PRETTY_PRINT = True

def debug_print(msg):
    if DEBUG_PRETTY_PRINT:
        gdb.write(f"[DEBUG] {msg}\n")

class MyPrinter:
    def __init__(self, val):
        debug_print(f"Initializing printer for {val.type}")
        self.val = val
    
    def to_string(self):
        debug_print(f"Converting {self.val.type} to string")
        return str(self.val)
```

**2. Exception Handling with Context**
```python
class RobustPrinter:
    def __init__(self, val):
        self.val = val
        self.typename = str(val.type)
    
    def to_string(self):
        try:
            # Your printer logic here
            return self._format_value()
        except Exception as e:
            return f"<Error in {self.typename} printer: {e}>"
    
    def children(self):
        try:
            # Your children logic here
            yield from self._get_children()
        except Exception as e:
            yield ("error", f"<Error getting children: {e}>")
```

**3. Type Inspection Utilities**
```python
def inspect_type(val):
    """Debug utility to inspect a GDB value's type information"""
    t = val.type
    print(f"Type: {t}")
    print(f"Code: {t.code}")
    print(f"Name: {t.name if hasattr(t, 'name') else 'None'}")
    print(f"Tag: {t.tag if hasattr(t, 'tag') else 'None'}")
    print(f"Sizeof: {t.sizeof}")
    
    if t.code == gdb.TYPE_CODE_STRUCT:
        print("Fields:")
        for field in t.fields():
            print(f"  {field.name}: {field.type}")
    
    # Template arguments for template types
    try:
        for i in range(10):  # Check first 10 template args
            arg = t.template_argument(i)
            print(f"Template arg {i}: {arg}")
    except RuntimeError:
        pass  # No more template arguments
```

## Advanced Custom Iterators for Complex Data Structures

### Tree Structure Iterator

```python
class TreeNodePrinter:
    """Pretty printer for tree nodes with custom traversal"""
    
    def __init__(self, val):
        self.val = val
        self.node_type = val.type
    
    def children(self):
        """Traverse tree in breadth-first order"""
        queue = [(self.val, "root")]
        visited = set()
        
        while queue:
            node, path = queue.pop(0)
            node_addr = int(node.address) if node.address else 0
            
            if node_addr in visited or node_addr == 0:
                continue
            visited.add(node_addr)
            
            # Yield current node
            yield (path, node["data"])
            
            # Add children to queue
            left = node["left"]
            right = node["right"]
            
            if left != 0:
                queue.append((left.dereference(), f"{path}.left"))
            if right != 0:
                queue.append((right.dereference(), f"{path}.right"))
    
    def to_string(self):
        return f"TreeNode at {self.val.address}"
    
    def display_hint(self):
        return "map"
```

### Hash Table Iterator with Bucket Traversal

```python
class HashTablePrinter:
    """Pretty printer for hash table with bucket iteration"""
    
    def __init__(self, val):
        self.val = val
        self.bucket_count = int(val["bucket_count"])
        self.buckets = val["buckets"]
    
    def children(self):
        """Iterate through all buckets and their chains"""
        count = 0
        for bucket_idx in range(self.bucket_count):
            bucket = self.buckets[bucket_idx]
            node = bucket["head"]
            
            chain_pos = 0
            while node != 0:
                key = node["key"]
                value = node["value"]
                yield (f"[{count}] bucket{bucket_idx}.{chain_pos}", 
                       f"{key} => {value}")
                
                node = node["next"]
                chain_pos += 1
                count += 1
    
    def to_string(self):
        size = int(self.val["size"])
        return f"HashTable<> with {size} elements in {self.bucket_count} buckets"
    
    def display_hint(self):
        return "map"
```

### Circular Buffer Iterator

```python
class CircularBufferIterator:
    """Iterator for circular buffer handling wraparound"""
    
    def __init__(self, buffer_start, capacity, head, tail, size):
        self.buffer_start = buffer_start
        self.capacity = capacity
        self.head = head
        self.tail = tail
        self.size = size
        self.index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= self.size:
            raise StopIteration
        
        # Calculate actual position with wraparound
        pos = (self.head + self.index) % self.capacity
        element = self.buffer_start[pos]
        
        result = f"[{self.index}]", element
        self.index += 1
        return result

class CircularBufferPrinter:
    def __init__(self, val):
        self.val = val
    
    def children(self):
        buffer_start = self.val["data"]
        capacity = int(self.val["capacity"])
        head = int(self.val["head"])
        tail = int(self.val["tail"])
        size = int(self.val["size"])
        
        return CircularBufferIterator(buffer_start, capacity, head, tail, size)
    
    def display_hint(self):
        return "array"
```

### Graph Iterator with Cycle Detection

```python
class GraphNodePrinter:
    """Pretty printer for graph with cycle detection"""
    
    def __init__(self, val):
        self.val = val
    
    def children(self):
        """Depth-first traversal with cycle detection"""
        visited = set()
        stack = [(self.val, "root", 0)]
        
        while stack:
            node, path, depth = stack.pop()
            node_addr = int(node.address) if node.address else 0
            
            if node_addr == 0:
                continue
                
            if node_addr in visited:
                yield (f"{path} (cycle)", f"-> already visited")
                continue
                
            visited.add(node_addr)
            yield (path, node["data"])
            
            # Limit depth to prevent infinite recursion
            if depth < 10:
                edges = node["edges"]
                edge_count = int(node["edge_count"])
                
                for i in range(edge_count):
                    edge_node = edges[i]
                    if edge_node != 0:
                        stack.append((edge_node.dereference(), 
                                    f"{path}.edge[{i}]", depth + 1))
    
    def display_hint(self):
        return "map"
```

## Performance Considerations

### Lazy Evaluation Techniques

**1. Lazy Field Access**
```python
class LazyContainerPrinter:
    """Printer that defers expensive operations until needed"""
    
    def __init__(self, val):
        self.val = val
        self._size = None
        self._data = None
    
    @property
    def size(self):
        """Lazy size calculation"""
        if self._size is None:
            self._size = int(self.val["size"])
        return self._size
    
    @property 
    def data(self):
        """Lazy data access"""
        if self._data is None:
            self._data = self.val["data"]
        return self._data
    
    def to_string(self):
        # Only access size, not data
        return f"Container with {self.size} elements"
    
    def children(self):
        # Only access data when children are requested
        for i in range(min(self.size, 100)):  # Limit to first 100
            yield f"[{i}]", self.data[i]
```

**2. Truncated Display for Large Containers**
```python
class LargeContainerPrinter:
    """Printer that handles large containers efficiently"""
    
    MAX_DISPLAY_SIZE = 50
    
    def __init__(self, val):
        self.val = val
        self.size = int(val["size"])
    
    def to_string(self):
        if self.size > self.MAX_DISPLAY_SIZE:
            return f"LargeContainer with {self.size} elements (showing first {self.MAX_DISPLAY_SIZE})"
        else:
            return f"LargeContainer with {self.size} elements"
    
    def children(self):
        display_size = min(self.size, self.MAX_DISPLAY_SIZE)
        data = self.val["data"]
        
        for i in range(display_size):
            yield f"[{i}]", data[i]
        
        if self.size > self.MAX_DISPLAY_SIZE:
            yield "...", f"({self.size - self.MAX_DISPLAY_SIZE} more elements)"
```

**3. Cached Property Pattern**
```python
def cached_property(func):
    """Decorator for caching expensive property calculations"""
    attr_name = f"_cached_{func.__name__}"
    
    def wrapper(self):
        if not hasattr(self, attr_name):
            setattr(self, attr_name, func(self))
        return getattr(self, attr_name)
    
    return property(wrapper)

class CachedPrinter:
    def __init__(self, val):
        self.val = val
    
    @cached_property
    def expensive_calculation(self):
        """This will only be calculated once"""
        # Simulate expensive operation
        result = 0
        for i in range(int(self.val["size"])):
            result += int(self.val["data"][i])
        return result
    
    def to_string(self):
        return f"Sum: {self.expensive_calculation}"
```

### Memory Access Optimization

**1. Bulk Memory Reading**
```python
def read_array_bulk(ptr, count, element_size):
    """Read array data in bulk for better performance"""
    if count == 0:
        return []
    
    # Read all data at once
    inferior = gdb.selected_inferior()
    total_bytes = count * element_size
    memory = inferior.read_memory(int(ptr), total_bytes)
    
    # Parse based on element size
    if element_size == 4:  # int32
        import struct
        return [struct.unpack('<i', memory[i:i+4])[0] for i in range(0, total_bytes, 4)]
    elif element_size == 8:  # int64
        import struct
        return [struct.unpack('<q', memory[i:i+8])[0] for i in range(0, total_bytes, 8)]
    else:
        # Fall back to individual access
        return [int(ptr[i]) for i in range(count)]

class BulkArrayPrinter:
    def __init__(self, val):
        self.val = val
        self.size = int(val["size"])
        self.data_ptr = val["data"]
    
    def children(self):
        if self.size > 1000:
            # Use bulk reading for large arrays
            element_type = self.data_ptr.type.target()
            element_size = element_type.sizeof
            values = read_array_bulk(self.data_ptr, self.size, element_size)
            for i, value in enumerate(values[:100]):  # Show first 100
                yield f"[{i}]", value
        else:
            # Use normal access for small arrays
            for i in range(self.size):
                yield f"[{i}]", self.data_ptr[i]
```

**2. String Optimization**
```python
class OptimizedStringPrinter:
    """Optimized string printer using lazy_string"""
    
    def __init__(self, val):
        self.val = val
    
    def to_string(self):
        # Use lazy_string for efficient string handling
        data = self.val["data"]
        size = int(self.val["size"])
        
        if size > 1000:
            # Truncate very long strings
            return data.lazy_string(length=100) + "... (truncated)"
        else:
            return data.lazy_string(length=size)
    
    def display_hint(self):
        return "string"
```

### Conditional Complexity

**1. Adaptive Display Based on Size**
```python
class AdaptivePrinter:
    """Printer that adapts complexity based on container size"""
    
    def __init__(self, val):
        self.val = val
        self.size = int(val["size"])
    
    def to_string(self):
        if self.size <= 10:
            # Show detailed info for small containers
            return f"SmallContainer[{self.size}]: {self._get_summary()}"
        elif self.size <= 100:
            # Medium detail for medium containers
            return f"MediumContainer[{self.size}]: first={self.val['data'][0]}, last={self.val['data'][self.size-1]}"
        else:
            # Minimal detail for large containers
            return f"LargeContainer[{self.size}]"
    
    def _get_summary(self):
        """Generate summary for small containers"""
        data = self.val["data"]
        return ", ".join(str(data[i]) for i in range(self.size))
    
    def children(self):
        # Adaptive children display
        if self.size <= 50:
            # Show all elements
            for i in range(self.size):
                yield f"[{i}]", self.val["data"][i]
        else:
            # Show samples
            data = self.val["data"]
            # First 10
            for i in range(10):
                yield f"[{i}]", data[i]
            yield "...", f"({self.size - 20} elements omitted)"
            # Last 10
            for i in range(self.size - 10, self.size):
                yield f"[{i}]", data[i]
```

## Testing Strategies for Pretty Printers

### Unit Testing Framework

**1. Test Infrastructure**
```python
# test_pretty_printers.py
import gdb
import unittest
from io import StringIO
import sys

class PrettyPrinterTestCase(unittest.TestCase):
    """Base class for pretty printer tests"""
    
    def setUp(self):
        # Compile test program if needed
        self.compile_test_program()
        # Start GDB session
        self.start_gdb_session()
    
    def compile_test_program(self):
        """Compile test C++ program"""
        import subprocess
        subprocess.run([
            "g++", "-g", "-O0", "test_program.cpp", "-o", "test_program"
        ], check=True)
    
    def start_gdb_session(self):
        """Initialize GDB session"""
        gdb.execute("file test_program")
        gdb.execute("break main")
        gdb.execute("run")
    
    def get_variable(self, var_name):
        """Get GDB variable"""
        return gdb.parse_and_eval(var_name)
    
    def get_pretty_string(self, var_name):
        """Get pretty-printed string representation"""
        val = self.get_variable(var_name)
        # Capture printer output
        old_stdout = sys.stdout
        sys.stdout = output = StringIO()
        try:
            gdb.execute(f"print {var_name}")
            return output.getvalue().strip()
        finally:
            sys.stdout = old_stdout
```

**2. Test Cases for Different Scenarios**
```python
class ContainerPrinterTest(PrettyPrinterTestCase):
    """Test cases for container pretty printers"""
    
    def test_empty_container(self):
        """Test empty container display"""
        output = self.get_pretty_string("empty_vector")
        self.assertIn("std::vector", output)
        self.assertIn("size=0", output)
    
    def test_small_container(self):
        """Test small container with known values"""
        output = self.get_pretty_string("small_vector")
        self.assertIn("[0] = 1", output)
        self.assertIn("[1] = 2", output)
        self.assertIn("[2] = 3", output)
    
    def test_large_container(self):
        """Test large container truncation"""
        output = self.get_pretty_string("large_vector")
        self.assertIn("truncated", output.lower())
        # Should not contain all 10000 elements
        self.assertLess(output.count("["), 100)
    
    def test_null_pointer(self):
        """Test null pointer handling"""
        output = self.get_pretty_string("null_ptr")
        self.assertIn("null", output.lower())
    
    def test_corrupted_data(self):
        """Test corrupted data handling"""
        # Set up corrupted data scenario
        gdb.execute("set var corrupted_container.size = -1")
        output = self.get_pretty_string("corrupted_container")
        # Should not crash, should show error message
        self.assertIn("error", output.lower())
```

**3. Test Data Generation**
```python
# test_data_generator.py
def generate_test_cpp():
    """Generate C++ test program with various data structures"""
    return '''
#include <vector>
#include <map>
#include <string>
#include <memory>

int main() {
    // Empty containers
    std::vector<int> empty_vector;
    std::map<int, std::string> empty_map;
    
    // Small containers
    std::vector<int> small_vector = {1, 2, 3, 4, 5};
    std::map<int, std::string> small_map = {{1, "one"}, {2, "two"}};
    
    // Large containers
    std::vector<int> large_vector;
    for (int i = 0; i < 10000; ++i) {
        large_vector.push_back(i);
    }
    
    // Nested structures
    std::vector<std::vector<int>> nested_vector = {{1, 2}, {3, 4, 5}};
    
    // Smart pointers
    auto shared_ptr = std::make_shared<int>(42);
    std::unique_ptr<int> unique_ptr = std::make_unique<int>(99);
    
    // Null pointers
    std::shared_ptr<int> null_shared;
    int* null_ptr = nullptr;
    
    // String types
    std::string short_string = "hello";
    std::string long_string(10000, 'x');
    
    return 0; // Breakpoint here
}
'''

def write_test_program():
    """Write test program to file"""
    with open("test_program.cpp", "w") as f:
        f.write(generate_test_cpp())
```

### Automated Testing

**1. Continuous Integration Test Script**
```python
#!/usr/bin/env python3
# ci_test.py
import subprocess
import sys
import os

def run_test_suite():
    """Run complete pretty printer test suite"""
    
    # Generate test program
    from test_data_generator import write_test_program
    write_test_program()
    
    # Compile test program
    result = subprocess.run([
        "g++", "-g", "-O0", "test_program.cpp", "-o", "test_program"
    ], capture_output=True, text=True)
    
    if result.returncode != 0:
        print(f"Compilation failed: {result.stderr}")
        return False
    
    # Run GDB with test script
    gdb_script = '''
set confirm off
file test_program
source pretty_printers.py
break main
run
python exec(open("test_pretty_printers.py").read())
quit
'''
    
    with open("test_script.gdb", "w") as f:
        f.write(gdb_script)
    
    result = subprocess.run([
        "gdb", "--batch", "--command=test_script.gdb"
    ], capture_output=True, text=True)
    
    # Analyze results
    if "FAILED" in result.stdout:
        print("Tests failed:")
        print(result.stdout)
        return False
    else:
        print("All tests passed!")
        return True

if __name__ == "__main__":
    success = run_test_suite()
    sys.exit(0 if success else 1)
```

**2. Regression Testing**
```python
class RegressionTest:
    """Test for regressions in pretty printer output"""
    
    def __init__(self, baseline_file="baseline_output.txt"):
        self.baseline_file = baseline_file
    
    def capture_baseline(self):
        """Capture current output as baseline"""
        output = self.run_all_tests()
        with open(self.baseline_file, "w") as f:
            f.write(output)
        print(f"Baseline captured to {self.baseline_file}")
    
    def check_regression(self):
        """Check for regressions against baseline"""
        current_output = self.run_all_tests()
        
        try:
            with open(self.baseline_file, "r") as f:
                baseline_output = f.read()
        except FileNotFoundError:
            print("No baseline found, capturing current output")
            self.capture_baseline()
            return True
        
        if current_output != baseline_output:
            print("REGRESSION DETECTED!")
            self.show_diff(baseline_output, current_output)
            return False
        else:
            print("No regression detected")
            return True
    
    def show_diff(self, baseline, current):
        """Show differences between baseline and current output"""
        import difflib
        diff = difflib.unified_diff(
            baseline.splitlines(keepends=True),
            current.splitlines(keepends=True),
            fromfile="baseline",
            tofile="current"
        )
        print("".join(diff))
```

### Mock and Stub Testing

**1. Mock GDB Values**
```python
class MockGDBValue:
    """Mock GDB value for testing without GDB"""
    
    def __init__(self, value_dict, typename):
        self._values = value_dict
        self._type = MockGDBType(typename)
    
    def __getitem__(self, key):
        if key in self._values:
            return self._values[key]
        raise gdb.error(f"No field named {key}")
    
    @property
    def type(self):
        return self._type
    
    @property
    def address(self):
        return MockGDBValue({"addr": 0x1000}, "void*")

class MockGDBType:
    def __init__(self, name):
        self.name = name
    
    def template_argument(self, index):
        if index == 0:
            return MockGDBType("int")
        raise RuntimeError("No template argument")

# Test usage
def test_printer_without_gdb():
    """Test printer logic without running GDB"""
    mock_vector = MockGDBValue({
        "size": MockGDBValue({"value": 3}, "size_t"),
        "data": MockGDBValue({"ptr": 0x2000}, "int*")
    }, "std::vector<int>")
    
    printer = VectorPrinter(mock_vector)
    result = printer.to_string()
    assert "size=3" in result
```

### Performance Testing

**1. Performance Benchmarks**
```python
import time
import statistics

def benchmark_printer(printer_class, test_values, iterations=100):
    """Benchmark pretty printer performance"""
    times = []
    
    for _ in range(iterations):
        start_time = time.time()
        
        for val in test_values:
            printer = printer_class(val)
            _ = printer.to_string()
            list(printer.children())  # Force evaluation
        
        end_time = time.time()
        times.append(end_time - start_time)
    
    return {
        'mean': statistics.mean(times),
        'median': statistics.median(times),
        'stdev': statistics.stdev(times) if len(times) > 1 else 0
    }

def performance_test_suite():
    """Run performance tests on all printers"""
    # Generate test data of various sizes
    small_data = generate_test_data(size=10)
    medium_data = generate_test_data(size=1000)
    large_data = generate_test_data(size=100000)
    
    printers = [VectorPrinter, MapPrinter, StringPrinter]
    
    for printer_class in printers:
        print(f"\nTesting {printer_class.__name__}:")
        
        for size, data in [("small", small_data), ("medium", medium_data), ("large", large_data)]:
            stats = benchmark_printer(printer_class, data)
            print(f"  {size}: {stats['mean']:.4f}s ± {stats['stdev']:.4f}s")
```

## Best Practices and Advanced Techniques

### Error Handling Patterns

**1. Graceful Degradation**
```python
class RobustPrinter:
    """Printer with comprehensive error handling"""
    
    def __init__(self, val):
        self.val = val
        self.errors = []
    
    def safe_access(self, field_path):
        """Safely access nested fields"""
        try:
            current = self.val
            for field in field_path.split('.'):
                current = current[field]
            return current
        except (gdb.error, KeyError) as e:
            self.errors.append(f"Cannot access {field_path}: {e}")
            return None
    
    def to_string(self):
        size = self.safe_access("size")
        if size is not None:
            return f"Container with {int(size)} elements"
        else:
            return f"Container (size unknown: {self.errors[-1]})"
    
    def children(self):
        if self.errors:
            yield ("errors", f"{len(self.errors)} access errors")
            for i, error in enumerate(self.errors):
                yield (f"error_{i}", error)
        
        # Try to show what we can
        data = self.safe_access("data")
        size = self.safe_access("size")
        
        if data is not None and size is not None:
            try:
                for i in range(min(int(size), 10)):
                    element = data[i]
                    yield (f"[{i}]", element)
            except Exception as e:
                yield ("partial_error", str(e))
```

**2. Version Compatibility**
```python
class VersionAwarePrinter:
    """Printer that adapts to different library versions"""
    
    def __init__(self, val):
        self.val = val
        self.version = self.detect_version()
    
    def detect_version(self):
        """Detect library version from type layout"""
        try:
            # Try newer field layout
            _ = self.val["new_field"]
            return "v2"
        except gdb.error:
            try:
                # Try older field layout
                _ = self.val["old_field"]
                return "v1"
            except gdb.error:
                return "unknown"
    
    def to_string(self):
        if self.version == "v2":
            return self.format_v2()
        elif self.version == "v1":
            return self.format_v1()
        else:
            return f"Unknown version of {self.val.type}"
    
    def format_v2(self):
        size = self.val["new_size_field"]
        return f"ModernContainer[{size}]"
    
    def format_v1(self):
        size = self.val["old_size_field"]
        return f"LegacyContainer[{size}]"
```

### Memory Safety

**1. Bounds Checking**
```python
class BoundsCheckedPrinter:
    """Printer with memory bounds checking"""
    
    def __init__(self, val):
        self.val = val
    
    def safe_array_access(self, array_ptr, index, max_size=1000):
        """Safely access array element with bounds checking"""
        try:
            if index < 0 or index >= max_size:
                return f"<index {index} out of safe bounds>"
            
            # Check if pointer is valid
            if int(array_ptr) == 0:
                return "<null pointer>"
            
            # Try to access the element
            element = array_ptr[index]
            return element
            
        except gdb.MemoryError:
            return f"<memory error at index {index}>"
        except Exception as e:
            return f"<error: {e}>"
    
    def children(self):
        data = self.val["data"]
        size = int(self.val["size"])
        
        # Sanity check on size
        if size < 0 or size > 10000:
            yield ("warning", f"Suspicious size: {size}")
            size = min(abs(size), 100)  # Limit to reasonable size
        
        for i in range(size):
            element = self.safe_array_access(data, i)
            yield (f"[{i}]", element)
```

**2. Circular Reference Detection**
```python
class CircularSafePrinter:
    """Printer that detects and handles circular references"""
    
    def __init__(self, val):
        self.val = val
        self.visited_addresses = set()
    
    def is_circular(self, ptr):
        """Check if pointer creates a circular reference"""
        if int(ptr) == 0:
            return False
        
        addr = int(ptr)
        if addr in self.visited_addresses:
            return True
        
        self.visited_addresses.add(addr)
        return False
    
    def children(self):
        self.visited_addresses.clear()
        self.visited_addresses.add(int(self.val.address))
        
        return self._traverse_children(self.val, "root", 0)
    
    def _traverse_children(self, node, path, depth):
        """Recursively traverse with circular detection"""
        if depth > 10:  # Prevent deep recursion
            yield (f"{path}_truncated", "<max depth reached>")
            return
        
        try:
            next_ptr = node["next"]
            if self.is_circular(next_ptr):
                yield (f"{path}_next", f"<circular reference to {hex(int(next_ptr))}>")
            elif int(next_ptr) != 0:
                next_node = next_ptr.dereference()
                yield (f"{path}_next", next_node["data"])
                yield from self._traverse_children(next_node, f"{path}_next", depth + 1)
                
        except Exception as e:
            yield (f"{path}_error", str(e))
```

### Advanced Display Hints

**1. Custom Display Hints**
```python
class AdvancedDisplayPrinter:
    """Printer with sophisticated display hints"""
    
    def __init__(self, val):
        self.val = val
        self.container_type = self.analyze_container_type()
    
    def analyze_container_type(self):
        """Analyze what kind of container this is"""
        type_name = str(self.val.type)
        
        if "map" in type_name.lower():
            return "associative"
        elif "set" in type_name.lower():
            return "set"
        elif "string" in type_name.lower():
            return "string"
        elif "array" in type_name.lower() or "vector" in type_name.lower():
            return "sequence"
        else:
            return "unknown"
    
    def display_hint(self):
        """Return appropriate display hint based on analysis"""
        mapping = {
            "associative": "map",
            "set": "array",  # Sets display well as arrays
            "string": "string", 
            "sequence": "array",
            "unknown": None
        }
        return mapping.get(self.container_type)
    
    def children(self):
        """Yield children appropriate for the container type"""
        if self.container_type == "associative":
            yield from self._map_children()
        elif self.container_type == "set":
            yield from self._set_children()
        else:
            yield from self._array_children()
    
    def _map_children(self):
        """Format as key-value pairs"""
        # Implementation for map-like containers
        pass
    
    def _set_children(self):
        """Format as unique values"""
        # Implementation for set-like containers
        pass
    
    def _array_children(self):
        """Format as indexed elements"""
        # Implementation for array-like containers
        pass
```

### Documentation and Maintenance

**1. Self-Documenting Printers**
```python
class DocumentedPrinter:
    """
    Pretty printer for MyType objects.
    
    Handles:
    - Empty containers (shows size 0)
    - Small containers (shows all elements)  
    - Large containers (shows truncated view)
    - Corrupted data (shows error messages)
    
    Display format: MyType[size] with optional element preview
    """
    
    def __init__(self, val):
        self.val = val
        self.size = self._get_size()
    
    def _get_size(self):
        """Get container size with error handling"""
        try:
            return int(self.val["size"])
        except:
            return -1  # Invalid size
    
    def to_string(self):
        """
        Returns string representation.
        Format: MyType[size] for valid containers
                MyType[invalid] for corrupted data
        """
        if self.size >= 0:
            return f"MyType[{self.size}]"
        else:
            return "MyType[invalid]"
    
    def children(self):
        """
        Yields container elements.
        - Shows all elements for size <= 20
        - Shows first 10 + last 10 for larger containers
        - Shows error info for corrupted containers
        """
        if self.size < 0:
            yield ("error", "Container size is invalid")
            return
        
        if self.size <= 20:
            # Show all elements
            for i in range(self.size):
                yield (f"[{i}]", self.val["data"][i])
        else:
            # Show abbreviated view
            for i in range(10):
                yield (f"[{i}]", self.val["data"][i])
            yield ("...", f"({self.size - 20} elements omitted)")
            for i in range(self.size - 10, self.size):
                yield (f"[{i}]", self.val["data"][i])
    
    def display_hint(self):
        """Returns 'array' for proper GDB formatting"""
        return "array"
```

**2. Registration Best Practices**
```python
def register_all_printers(objfile=None):
    """
    Register all pretty printers for the project.
    
    Args:
        objfile: GDB objfile to register printers for, or None for global
    """
    
    # Create collection with descriptive name
    pp = gdb.printing.RegexpCollectionPrettyPrinter("my_project_v1.0")
    
    # Register printers with clear, specific patterns
    printers = [
        # Core containers
        ("MyVector", r"^my_project::vector<.*>$", MyVectorPrinter),
        ("MyMap", r"^my_project::map<.*>$", MyMapPrinter), 
        ("MyString", r"^my_project::string$", MyStringPrinter),
        
        # Smart pointers
        ("MySharedPtr", r"^my_project::shared_ptr<.*>$", MySharedPtrPrinter),
        ("MyUniquePtr", r"^my_project::unique_ptr<.*>$", MyUniquePtrPrinter),
        
        # Specialized types
        ("MyOptional", r"^my_project::optional<.*>$", MyOptionalPrinter),
    ]
    
    for name, pattern, printer_class in printers:
        try:
            pp.add_printer(name, pattern, printer_class)
            print(f"Registered {name} printer")
        except Exception as e:
            print(f"Failed to register {name} printer: {e}")
    
    # Register the collection
    if objfile is None:
        objfile = gdb.current_objfile()
    
    gdb.printing.register_pretty_printer(objfile, pp)
    print(f"Registered {len(printers)} pretty printers")

# Auto-register when module is imported
if __name__ != "__main__":
    try:
        register_all_printers()
    except Exception as e:
        print(f"Failed to auto-register printers: {e}")
```

## Quick Reference and Summary

### Essential GDB Pretty Printer Checklist

**1. Basic Structure**
```python
class MyPrinter:
    def __init__(self, val):
        self.val = val
    
    def to_string(self):
        return "Brief description"
    
    def children(self):
        yield ("key", value)
    
    def display_hint(self):
        return "array"  # or "map" or "string"
```

**2. Registration Pattern**
```python
pp = gdb.printing.RegexpCollectionPrettyPrinter("project_name")
pp.add_printer("TypeName", r"^namespace::TypeName<.*>$", PrinterClass)
gdb.printing.register_pretty_printer(gdb.current_objfile(), pp)
```

**3. Common Patterns**
- **Type cleanup**: `val.type.unqualified().strip_typedefs()`
- **Template args**: `val.type.template_argument(0)`
- **Safe access**: `try/except gdb.error`
- **Memory reading**: `inferior.read_memory(addr, size)`
- **String handling**: `data.lazy_string(length=size)`

### Performance Tips
- ✅ Use lazy evaluation for expensive operations
- ✅ Truncate large containers (limit to ~50-100 elements)
- ✅ Cache expensive calculations
- ✅ Use bulk memory reads for arrays
- ❌ Don't access all elements in `to_string()`
- ❌ Don't perform expensive operations in `__init__`

### Error Handling Best Practices
- ✅ Always handle `gdb.error` exceptions
- ✅ Provide meaningful error messages
- ✅ Degrade gracefully on corrupted data
- ✅ Check for null pointers
- ✅ Validate array bounds
- ❌ Don't let exceptions crash GDB
- ❌ Don't assume data structure integrity

### Display Hints Guide
- `"array"` - For sequences, lists, vectors, sets
- `"map"` - For associative containers, key-value pairs  
- `"string"` - For string-like objects
- `None` - For simple objects, custom formatting

### Testing Strategy
- ✅ Test with empty containers
- ✅ Test with small containers (1-10 elements)
- ✅ Test with large containers (1000+ elements)
- ✅ Test with null/invalid pointers
- ✅ Test with corrupted data
- ✅ Test performance with large datasets
- ✅ Test across different compiler versions

### Debugging Printer Issues
```python
# Enable debug output
DEBUG = True
def debug_print(msg):
    if DEBUG:
        gdb.write(f"[DEBUG] {msg}\n")

# Inspect type information
def inspect_type(val):
    print(f"Type: {val.type}")
    print(f"Fields: {[f.name for f in val.type.fields()]}")

# Check printer registration
(gdb) info pretty-printer

# Enable Python stack traces
(gdb) set python print-stack full
```

### Common Gotchas
1. **Reference types**: Use `real_type()` to strip references and const
2. **Template instantiations**: Match with `^TypeName<.*>$` not exact types
3. **Memory errors**: Always check for null pointers before dereferencing
4. **Large data**: Truncate display to prevent GDB hanging
5. **Version differences**: Handle different library versions gracefully
6. **Circular references**: Track visited addresses to prevent infinite loops

### Resources for Further Learning
- **LLVM Pretty Printers**: https://github.com/llvm/llvm-project/tree/main/llvm/utils/gdb-scripts/
- **GCC Pretty Printers**: https://github.com/gcc-mirror/gcc/tree/main/gcc/gdbhooks.py
- **Boost Pretty Printers**: https://github.com/ruediger/Boost-Pretty-Printer
- **GDB Python API**: https://sourceware.org/gdb/onlinedocs/gdb/Python-API.html
- **Pretty Printer Documentation**: https://sourceware.org/gdb/onlinedocs/gdb/Pretty-Printing.html

### Example Complete Printer

```python
class ExamplePrinter:
    """Complete example showing all best practices"""
    
    def __init__(self, val):
        self.val = val
        self._size = None
        self._data = None
    
    @property
    def size(self):
        if self._size is None:
            try:
                self._size = int(self.val["size"])
                if self._size < 0 or self._size > 1000000:
                    self._size = 0  # Sanity check
            except:
                self._size = 0
        return self._size
    
    @property
    def data(self):
        if self._data is None:
            try:
                self._data = self.val["data"]
            except:
                self._data = None
        return self._data
    
    def to_string(self):
        return f"ExampleContainer[{self.size}]"
    
    def children(self):
        if self.data is None:
            yield ("error", "Cannot access data")
            return
        
        display_count = min(self.size, 50)
        for i in range(display_count):
            try:
                yield (f"[{i}]", self.data[i])
            except:
                yield (f"[{i}]", "<error>")
        
        if self.size > 50:
            yield ("...", f"({self.size - 50} more elements)")
    
    def display_hint(self):
        return "array"

# Registration
def register_example_printer():
    pp = gdb.printing.RegexpCollectionPrettyPrinter("example")
    pp.add_printer("ExampleContainer", 
                   r"^ExampleContainer<.*>$", 
                   ExamplePrinter)
    gdb.printing.register_pretty_printer(gdb.current_objfile(), pp)

register_example_printer()
```

This comprehensive reference covers all the essential aspects of GDB pretty printer development, from basic implementation to advanced techniques, testing strategies, and debugging tips.